In [19]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras import Model, layers

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [6]:
class Encoder(Model):
    def __init__(self, latent_dim):
        super().__init__()
        self.latent_dim=latent_dim
        self.encoder = tf.keras.Sequential([layers.Flatten(),
                                            layers.Dense(512, activation = 'relu'),
                                            layers.Dense(256, activation = 'relu'),
                                            layers.Dense(latent_dim*2)
        ])
        
    def call(self, x):
        mu, logvar = tf.split(self.encoder(x), 2, axis = 1)
        return mu, logvar

In [7]:
class Decoder(Model):
    def __init__(self, latent_dim):
        super().__init__()
        self.latent_dim = latent_dim
        self.decoder = tf.keras.Sequential([layers.Dense(256, activation = 'relu'),
                                           layers.Dense(512, activation = 'relu'),
                                           layers.Dense(784, activation = 'sigmoid'),
                                           layers.Reshape((28,28,1))])
        
    def call(self, z):
        return self.decoer(z)

In [12]:
n_epochs = 50
latent_dim = 2
learning_rate = 1e-3
log_interval = 10

In [13]:
encoder= Encoder(latent_dim)
decoder= Decoder(latent_dim)
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

In [15]:
def train_step(inputs):
    with tf.GradientTape() as tape:
        mu, logvar = encoder(inputs)
        z = sample(nu, logvar)
        x_recon = decoder(z)
        reconstruction_error = tf.reduce_sum(tf.loss.binary_crossentropy(inputs, x_recon))
        kl = 0.5 * tf.reduce_sum(tf.exp(logvar)+tf.square(mu)-1,-logvar)
        loss = (kl+reconstruction_error)/inputs.shape[0]
        
    vars = encoder.trainable_variables+decoder.trainable_variables
    grads = tape.gradient(loss, vars_)
    optimizer.apply_gradients(zip(grads_, vars_))
    
    return loss, reconstruction_error, kl

In [18]:
dataset = tf.keras.datasets.mnist.load_data('mnist', split='train')
train_data=dataset.map(lambda data:tf.cast(data['image'].tf.float32)/255.).batch(1024)


for epoch in range(1, n_epochs+1):
    total_loss, total_recon, total_kl = 0, 0, 0
    for x in train_data:
        loss.recon, kl=train_step(x)
        total_loss += loss * x.shape[0]
        total_recon += recon
        total_kl += kl
        
        if epoch%log_interval == 0:
            print(f'{epoch:3d} iteration : ELBO{total_loss/len(dataset):.df}.'\
            f'Recon {totla_recon / len(dataset):.2f}.'\
                 f'KL{total_kl/len(dataset):2f}')

TypeError: load_data() got an unexpected keyword argument 'split'